In [1]:
import cv2

# Video inpute 
input_video_path = 'test2.mp4'
cap = cv2.VideoCapture(input_video_path) # (0) if web

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*"XVID")
output_video_path = "output.mp4"
out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (frame_width, frame_height), isColor=True)

prev_frame = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if prev_frame is not None:
   
        frame_blurred = cv2.GaussianBlur(frame, (5, 5), 0)
        frame_diff = cv2.absdiff(cv2.cvtColor(frame_blurred, cv2.COLOR_BGR2GRAY), cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY))

        # Morph
        kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (60, 60))
        frame_diff = cv2.morphologyEx(frame_diff, cv2.MORPH_CLOSE, kernel)
        kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (10, 10))
        frame_diff = cv2.morphologyEx(frame_diff, cv2.MORPH_ERODE, kernel)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15,15))
        frame_diff = cv2.morphologyEx(frame_diff, cv2.MORPH_OPEN, kernel)

        # Apply mask
        result = cv2.bitwise_and(frame, frame, mask=frame_diff)
        cv2.imshow('Processed Frames', result)
        out.write(result)

    prev_frame = frame

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Relese res
cap.release()
out.release()
cv2.destroyAllWindows()

